# Universal Thermal Comfort Index (UTCI) classes
This notebook calculates the number of nights per year in each UTCI class, based on the UTCI hourly data available at the Copernicus climate data store, in NetCDF format. The values were calculated for the nights: The maximum temperatures between 18h-6h were used.
- UTCI definition: For any combination of air temperature, wind, radiation and humidity, the Universal thermal climate index is defined as the air temperature of a reference outdoor environment that would elicit in the human body the same physiological model’s response (sweat production, shivering, skin wettedness, skin blood flow and rectal, mean skin and face temperatures) as the actual environment.
- Data source: https://cds.climate.copernicus.eu/cdsapp#!/dataset/derived-utci-historical?tab=overview
- UTCI classes:
| UTCI value \[°C\] | UTCI class              |
|----------------|-------------------------|
| >46            | Extreme Heat Stress     |
| 38-46          | Very Strong Heat Stress |
| 32-38          | Strong Heat Stress      |
| 26-32          | Moderate Heat Stress    |
| 9-26           | No Thermal Stress       |
| 0-9            | Slight Cold Stress      |
| -13-0          | Moderate Cold Stress    |
| -27--13        | Strong Cold Stress      |
| -40--27        | Very Strong Cold Stress |
| <-40           | Extreme Cold Stress     |

## Workflow
1. Download yearly data from CDS. Unzip the files in `utci_hourly_path/<year>`. There is one NetCDF file per day
2. Subset and re-chunk the data into one file and save it in `utci_yearly_path/<year>`
3. Compute the classes
4. Save results to database

## Important note
Since the data is in NetCDF format, which is not cloud-optimized, it is highly recommended to download the data locally and run this notebook in a your local machine.

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
from configparser import ConfigParser
import sqlalchemy as sa # conection to the database
from datetime import datetime, timedelta

year = '2018'
utci_hourly_path = f'N:/C2205_FAIRiCUBE/f02_data/d006_climate_data/f01_thermal_comfort_index/{year}'
utci_yearly_path = f'N:/C2205_FAIRiCUBE/f02_data/d006_climate_data/f01_thermal_comfort_index/yearly/{year}'


## Open hourly data
Open all data for one year, convert from Kelvin to Celsius, and save to unique file

In [ ]:
data_one_year_df = xr.open_mfdataset(utci_hourly_path+"/*.nc")

utci = data_one_year_df.utci -273.15
utci.attrs = data_one_year_df.utci.attrs
utci.attrs["units"] = "C"

In [ ]:
utci

## Get cities and coordinates and filter UTCI data
### Connect to database

In [ ]:
from sqlalchemy import create_engine, text

# helper function to connect to the database
def config(filename, section='postgresql'):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)

    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception(
            'Section {0} not found in the {1} file'.format(section, filename))

    return db

keys = config(filename='database.ini')

POSTGRESQL_SERVER_NAME=keys['host']
PORT=                  keys['port']
Database_name =        keys['database']
USER =                 keys['user']
PSW =                  keys['password']
##################################################
                                   
engine_postgresql = sa.create_engine('postgresql://'+USER+':'+PSW+ '@'+POSTGRESQL_SERVER_NAME+':'+str(PORT)+ '/' + Database_name)
print (engine_postgresql)


In [ ]:
with engine_postgresql.begin() as conn:
    query = text("""SELECT urau_code, _wgs84y, _wgs84x, time_zone_offset
    FROM lut.l_city_urau2021;""")
    city_center_df = pd.read_sql_query(query, conn)
    
city_center_df_r = city_center_df#[60:62] #use this to create dataset subset

# get city coordinates
# lonlat_list =[["NL005C", 4.640960, 52.113299], ["NL006C", 5.384670, 52.173656], ["NL007C", 5.921886, 52.189884]]

lon_list = city_center_df_r["_wgs84x"].values.tolist()
lat_list = city_center_df_r["_wgs84y"].values.tolist()
city_list = city_center_df_r["urau_code"].values.tolist()
target_lon = xr.DataArray(lon_list, dims="city", coords={"city": city_list})
target_lat = xr.DataArray(lat_list, dims="city", coords={"city": city_list})
time_zone_offset = xr.DataArray(city_center_df_r['time_zone_offset'], dims="city", coords={"city": city_list})

## next filter dataframe by city:

data_one_year_df_cities = utci.sel(
    lon=target_lon, 
    lat=target_lat, method="ffill")

### Save UTCI data subset to one file
This speeds up further computations because avoids opening all daily NetCDF files for the entire data extent

In [ ]:
data_one_year_df_cities_tz = xr.merge([data_one_year_df_cities,time_zone_offset])
# save city dataset
data_one_year_df_cities_tz.to_netcdf(path = utci_yearly_path+f"/{year}_cities.nc")
data_one_year_df_cities_tz

### Load data and conver to Dask DataFrame

In [ ]:
load_data = xr.open_mfdataset(utci_yearly_path+f"/{year}_cities.nc")
load_data_df = load_data.chunk({'city': 100, 'time': -1}).to_dask_dataframe()

## Compute nighttime max UTCI 
### Helper functions

In [ ]:
def daytime(x):
    time_local = x.time+timedelta(hours=x.timezone)
    if(time_local.hour >= 6 and time_local.hour < 18):
        return 1
    else:
        return 0

def night_date(x):
    # assign previous day date to nighttime hours [0..end_night]
    # first adjust to local time
    time_local = x.time+timedelta(hours=x.timezone)
    if(time_local.hour < 6):
        # date = x.time - timedelta(days=1)
        return x.time - timedelta(days=1)
    else:
        return x.time
    
def classify_utci_df(x):
    utci_value = x.utci
    if utci_value > 46:           return "utci_class_extreme_heat_stress"
    elif 38 <= utci_value <= 46:  return "utci_class_very_strong_heat_stress"
    elif 32 <= utci_value < 38:   return "utci_class_strong_heat_stress"
    elif 26 <= utci_value < 32:   return "utci_class_moderate_heat_stress"
    elif 9 <= utci_value < 26:    return "utci_class_no_thermal_stress"
    elif 0 <= utci_value < 9:     return "utci_class_slight_cold_stress"
    elif -13 <= utci_value < 0:   return "utci_class_moderate_cold_stress"
    elif -27 <= utci_value < -13: return "utci_class_strong_cold_stress"
    elif -40 <= utci_value < -27: return "utci_class_very_strong_cold_stress"
    else:                         return "utci_class_extreme_cold_stress"

### Start Dask cluster

In [ ]:
from dask.distributed import Client, performance_report
client = Client()  # Connect to distributed cluster and override default
client

### Compute

In [ ]:
load_data_df['daytime'] = load_data_df.apply(daytime, axis=1, meta=(None, 'int'))
load_data_df['time_shifted'] = load_data_df.apply(night_date, axis=1, meta=(None, 'datetime64[ns]'))
load_data_df['date'] = load_data_df.time_shifted.dt.date
utci_max = load_data_df.groupby(['city', 'daytime', 'date']).max()
utci_max['utci_class'] = utci_max.apply(classify_utci_df, axis=1, meta=(None, 'string'))
with performance_report(filename='utci_classes_2022_df.html'):
    utci_max_c = utci_max.compute()


In [ ]:
df = utci_max_c[utci_max_c.daytime == 0][['city', 'date', 'utci', 'utci_class']]
df_count = df.groupby(['city', 'utci_class']).count()
df_count

## Export indicator to database

In [101]:
##################################
## cu_* tables mandatory columns:
# city_code,
# parameter, 
# parameter_id, 
# parameter_value, 
# year, 
# city_code_version, 
# lineage, 
# datasource

df_count.reset_index(inplace=True)
df_count.rename(columns={'city': 'city_code',
                        'utci': 'parameter_value',
                        'utci_class': 'parameter_id'})

# new columns:

df_count['year'] = year
df_count['city_code_version'] = 'ua_2021'
df_count['parameter'] = 'Number of nights per year in UTCI class: ' + df_count['parameter_id']
df_count['lineage'] = 'The night hours were first extracted from the hourly values: from locally 18h to locally 6h. In the next step, the UTCI temps were calculated within the NIGHT and transferred to UTCI classes. The number of days (nights) for each class was then calculated for each year.'
df_count['datasource']= 'https://cds.climate.copernicus.eu/cdsapp#!/dataset/derived-utci-historical?tab=overview'
df_count

send table to SQL:
export done


In [98]:
##################################################
name_of_table = 'cu_city_utci_indicator'
schmema_name ='cube'
##################################################
df_count.to_sql(name_of_table, engine_postgresql,  schema=schmema_name,if_exists='replace')

,utci_class,year,city_code,city_code_version,parameter,parameter_value,lineage,datasource
0,Moderate Cold Stress,2021,AT001C1,ua_2021,Number days per year of UTCI-class (derived fr...,141,The night hours were first extracted from the ...,cds.climate.copernicus.eu -> https://cds.clima...
1,Moderate Heat Stress,2021,AT001C1,ua_2021,Number days per year of UTCI-class (derived fr...,95,The night hours were first extracted from the ...,cds.climate.copernicus.eu -> https://cds.clima...
2,No Thermal Stress,2021,AT001C1,ua_2021,Number days per year of UTCI-class (derived fr...,276,The night hours were first extracted from the ...,cds.climate.copernicus.eu -> https://cds.clima...
3,Slight Cold Stress,2021,AT001C1,ua_2021,Number days per year of UTCI-class (derived fr...,182,The night hours were first extracted from the ...,cds.climate.copernicus.eu -> https://cds.clima...
4,Strong Cold Stress,2021,AT001C1,ua_2021,Number days per year of UTCI-class (derived fr...,9,The night hours were first extracted from the ...,cds.climate.copernicus.eu -> https://cds.clima...
...,...,...,...,...,...,...,...,...
5680,Moderate Heat Stress,2021,UK586C1,ua_2021,Number days per year of UTCI-class (derived fr...,21,The night hours were first extracted from the ...,cds.climate.copernicus.eu -> https://cds.clima...
5681,No Thermal Stress,2021,UK586C1,ua_2021,Number days per year of UTCI-class (derived fr...,338,The night hours were first extracted from the ...,cds.climate.copernicus.eu -> https://cds.clima...
5682,Slight Cold Stress,2021,UK586C1,ua_2021,Number days per year of UTCI-class (derived fr...,228,The night hours were first extracted from the ...,cds.climate.copernicus.eu -> https://cds.clima...
5683,Strong Cold Stress,2021,UK586C1,ua_2021,Number days per year of UTCI-class (derived fr...,4,The night hours were first extracted from the ...,cds.climate.copernicus.eu -> https://cds.clima...
